In [153]:
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import string
import re

In [154]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")

    # General replacements
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    
    
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [155]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTFigure, LTTextBox
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage, PDFTextExtractionNotAllowed
from pdfminer.pdfparser import PDFParser
import nltk.data

filename = 'HarryPotter1.pdf' 
text = ""
stack = ""

with open(filename, 'rb') as f:
    parser = PDFParser(f)
    doc = PDFDocument(parser)
    page = list(PDFPage.create_pages(doc))[0]
    rsrcmgr = PDFResourceManager()
    device = PDFPageAggregator(rsrcmgr, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    interpreter.process_page(page)
    layout = device.get_result()
    
    for obj in layout:
        if isinstance(obj, LTTextBox):
            text += obj.get_text()
            
        elif isinstance(obj, LTFigure):
            stack += list(obj)

print(stack)

In [156]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open(filename, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

text = output_string.getvalue()


In [157]:
sentences = split_into_sentences(text)
smallSentence = []
smallPunctuation = []
final = []

for sentence in sentences:
    res = sum([i.strip(string.punctuation).isalpha() for i in sentence.split()])

    if res<9 and res >1:
        sentenceWOPuct = sentence.translate(str.maketrans('', '', string.punctuation))
        sentenceList = re.split(r"\s+", sentenceWOPuct)
        punctuationList = re.split(r"\w+", sentence)
        smallPunctuation.append(punctuationList)

        for index, punctuationSign in enumerate(punctuationList):
            if punctuationSign == '.' or punctuationSign == '?' or punctuationSign == ',':
                sentenceList.append(punctuationSign)
        smallSentence.append(sentenceList)

print(smallSentence)

[['What', 'could', 'he', 'have', 'been', 'thinking', 'of', '?'], ['Mr', 'Dursley', 'blinked', 'and', 'stared', 'at', 'the', 'cat', '.'], ['It', 'stared', 'back', '.'], ['People', 'in', 'cloaks', '.'], ['He', 'supposed', 'this', 'was', 'some', 'stupid', 'new', 'fashion', '.'], ['They', 'were', 'whispering', 'excitedly', 'together', '.'], ['The', 'nerve', 'of', 'him'], ['yes', 'that', 'would', 'be', 'it', '.'], ['Mr', 'Dursley', 'however', 'had', 'a', 'perfectly', 'normal', 'owlfree', 'morning', '.'], ['He', 'yelled', 'at', 'five', 'different', 'people', '.'], ['He', 'eyed', 'them', 'angrily', 'as', 'he', 'passed', '.'], ['He', 'didnt', 'know', 'why', 'but', 'they', 'made', 'him', 'uneasy', '.'], ['Fear', 'flooded', 'him', '.'], ['no', 'he', 'was', 'being', 'stupid', '.'], ['Potter', 'wasnt', 'such', 'an', 'unusual', 'name', '.'], ['Hed', 'never', 'even', 'seen', 'the', 'boy', '.'], ['It', 'might', 'have', 'been', 'Harvey', '.'], ['Or', 'Harold', '.'], ['but', 'all', 'the', 'same', 'thos